<a href="https://colab.research.google.com/github/Gbell26/GestureBot/blob/main/Gesture_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/keypoint_data.json" "/content/"

In [ ]:
with open('keypoint_data.json', 'r') as f:
    data = json.load(f)

In [ ]:
classes = []
keypoints = []

In [ ]:
for image in data:
    classes.append(image['class'])
    keypoints.append(image['keypoints'][0][0][0])

In [ ]:
classes = np.array(classes)
keypoints = np.array(keypoints)

In [ ]:
print(classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 

In [ ]:
print(keypoints)

[[0.36780792 0.69278157 0.45935553 ... 0.43662706 0.33217093 0.39173117]
 [0.41362903 0.70046592 0.51438504 ... 0.43435627 0.31934199 0.38216403]
 [0.40454108 0.70551229 0.50429261 ... 0.43380874 0.33289084 0.3942363 ]
 ...
 [0.44017041 0.5994221  0.47358847 ... 0.40305978 0.47652239 0.44571793]
 [0.44450259 0.62641293 0.44519833 ... 0.4316633  0.54243135 0.41886827]
 [0.39736015 0.64936107 0.45989296 ... 0.45178083 0.43992689 0.4569259 ]]


In [ ]:
print(keypoints.shape)

(827, 42)


In [ ]:
print(classes.shape)

(827,)


In [ ]:
#split dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(keypoints, classes, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)

(661, 42)


In [ ]:
print(X_val)

[[0.55475175 0.68858075 0.4707011  ... 0.39915887 0.6397602  0.34549153]
 [0.45519614 0.68313694 0.50108618 ... 0.41138694 0.55877161 0.39285696]
 [0.37518594 0.63843578 0.45973274 ... 0.39681932 0.47735772 0.37956402]
 ...
 [0.53869516 0.69811666 0.45664784 ... 0.38151401 0.63322461 0.32917637]
 [0.49049628 0.70676041 0.47975621 ... 0.39006567 0.52617753 0.38352439]
 [0.45726129 0.6800347  0.43850267 ... 0.41782546 0.59502053 0.39064515]]


In [ ]:
NUM_CLASSES = 6

In [ ]:
inputs = layers.Input(shape=(42,))

x = inputs
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(NUM_CLASSES, activation = 'softmax')(x)

model = keras.Model(inputs, outputs, name="Gesture_Classifier")

model.summary()

Model: "Gesture_Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 42)]              0         
                                                                 
 flatten (Flatten)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 128)               5504      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)         

In [ ]:
callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
r = model.fit(X_train, y_train, epochs=500, validation_data=(X_val, y_val),callbacks=[callback],)

Epoch 1/500
21/21 [==============================] - 1s 16ms/step - loss: 1.7402 - accuracy: 0.2209 - val_loss: 1.6856 - val_accuracy: 0.2711
Epoch 2/500
21/21 [==============================] - 0s 4ms/step - loss: 1.6981 - accuracy: 0.2300 - val_loss: 1.6834 - val_accuracy: 0.2108
Epoch 3/500
21/21 [==============================] - 0s 5ms/step - loss: 1.6805 - accuracy: 0.2814 - val_loss: 1.6703 - val_accuracy: 0.2711
Epoch 4/500
21/21 [==============================] - 0s 5ms/step - loss: 1.6892 - accuracy: 0.2496 - val_loss: 1.6701 - val_accuracy: 0.2831
Epoch 5/500
21/21 [==============================] - 0s 4ms/step - loss: 1.6584 - accuracy: 0.2738 - val_loss: 1.6525 - val_accuracy: 0.2892
Epoch 6/500
21/21 [==============================] - 0s 5ms/step - loss: 1.6479 - accuracy: 0.2874 - val_loss: 1.6394 - val_accuracy: 0.2952
Epoch 7/500
21/21 [==============================] - 0s 5ms/step - loss: 1.6483 - accuracy: 0.2844 - val_loss: 1.6293 - val_accuracy: 0.2771
Epoch 8/500


In [ ]:
!mkdir Gesture_ClassifierV2

In [ ]:
#save the model
model.save(
    "/content/Gesture_ClassifierV2"
)

In [ ]:
#convert to tflite
try:
  converter = tf.lite.TFLiteConverter.from_saved_model('/content/Gesture_ClassifierV2')
  tflite_model = converter.convert()

  with open('/content/Gesture_ClassifierV2/Gesture_ClassifierV2.tflite', 'wb') as f:
    f.write(tflite_model)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from tensorflow.lite.python.interpreter import Interpreter
classifier_interpreter = Interpreter(model_path='/content/Gesture_ClassifierV2/Gesture_ClassifierV2.tflite')
classifier_interpreter.allocate_tensors()

classifier_input_details = classifier_interpreter.get_input_details()
classifier_output_details = classifier_interpreter.get_output_details()

print(classifier_input_details)
print(classifier_output_details)

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([ 1, 42], dtype=int32), 'shape_signature': array([-1, 42], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 15, 'shape': array([1, 6], dtype=int32), 'shape_signature': array([-1,  6], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
%%bash
cat <<EOF >> /content/labelmap.txt
Open_Hand
Pointer_Thumb
Middle_Thumb
Ring_Thumb
Pinky_Thumb
None
EOF

In [ ]:
!cp /content/labelmap.txt /content/Gesture_ClassifierV2

In [ ]:
from google.colab import files

!zip -r '/content/Gesture_ClassifierV2' '/content/Gesture_ClassifierV2'

  adding: content/Gesture_ClassifierV2/ (stored 0%)
  adding: content/Gesture_ClassifierV2/fingerprint.pb (stored 0%)
  adding: content/Gesture_ClassifierV2/variables/ (stored 0%)
  adding: content/Gesture_ClassifierV2/variables/variables.data-00000-of-00001 (deflated 24%)
  adding: content/Gesture_ClassifierV2/variables/variables.index (deflated 62%)
  adding: content/Gesture_ClassifierV2/saved_model.pb (deflated 88%)
  adding: content/Gesture_ClassifierV2/keras_metadata.pb (deflated 89%)
  adding: content/Gesture_ClassifierV2/assets/ (stored 0%)
  adding: content/Gesture_ClassifierV2/labelmap.txt (deflated 22%)
  adding: content/Gesture_ClassifierV2/Gesture_ClassifierV2.tflite (deflated 9%)


In [ ]:
#download the model
files.download("/content/Gesture_ClassifierV2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>